# **Imports**

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
!pip install transformers accelerate bitsandbytes streamlit nltk rouge --upgrade
import torch
import streamlit as st
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score


In [2]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

# LLama **1B**

In [1]:
!pip install -U bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision
    bnb_4bit_compute_dtype="float16",  # Compute with float16
    bnb_4bit_use_double_quant=True,  # Further reduce memory
)

# Load the tokenizer and model with quantization
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    quantization_config=bnb_config,
    device_map="auto"  # Distributes layers across available GPUs
)

print("Model loaded successfully with 4-bit quantization!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model loaded successfully with 4-bit quantization!


In [7]:
import torch
document = input("Enter your document text for paraphrasing:\n")

def split_into_chunks(text, max_tokens=750):
    tokens = tokenizer(text, return_tensors="pt", truncation=False).input_ids[0]
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))
    return chunks

paraphrased_chunks = []
chunks = split_into_chunks(document)

for i, chunk in enumerate(chunks):
    if i > 0:
        chunk = chunks[i-1][-100:] + "\n" + chunk

    prompt = (
            "Paraphrase the following text accurately while preserving:\n"
            "- The original meaning, tone, and style.\n"
            "- Logical sentence structure and clarity.\n"
            "- All critical details and imagery.\n\n"
            "Return ONLY the paraphrased text without any explanations, prefixes, or suffixes.\n\n"
            f"### Original Text:\n{chunk}\n\n"
            "### Paraphrased Text:"
        )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=600).to(device)
    output = model.generate(
        **inputs,
        max_length=1000,
        do_sample=True,
        temperature=1,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

    paraphrased = tokenizer.decode(output[0], skip_special_tokens=True)
    paraphrased = paraphrased.split("Paraphrased Text:")[-1].strip()
    paraphrased_chunks.append(paraphrased)

paraphrased_document = "\n".join(paraphrased_chunks)

print("\n--- Paraphrased Document ---\n")
print(paraphrased_document)

Enter your document text for paraphrasing:
In the vast tapestry of the universe, where galaxies swirl in spirals of cosmic ballet and stars are born and extinguished in cataclysmic brilliance, humankind finds itself perched on a small, pale blue dot suspended in a sunbeam. Our world, a fragile orb teeming with life, floats in the immensity of space, surrounded by silent stars and the cold, unfeeling void. Despite our seemingly insignificant presence in the cosmos, our minds possess an insatiable curiosity—a relentless drive to explore, understand, and question the very fabric of reality.  Science, the great tool of inquiry, has carried us far. From the discovery of subatomic particles to the observation of distant quasars, we have unraveled secrets once hidden in the darkest corners of ignorance. Yet, as each question is answered, new ones arise, cascading like ripples on a pond. What lies beyond the observable universe? Are there other dimensions folded into the creases of spacetime? 

# LLama **3B**

In [8]:
# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision
    bnb_4bit_compute_dtype="float16",  # Compute with float16
    bnb_4bit_use_double_quant=True,  # Further reduce memory
)

# Load the tokenizer and model with quantization
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    quantization_config=bnb_config,
    device_map="auto"  # Distributes layers across available GPUs
)

print("Model loaded successfully with 4-bit quantization!")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model loaded successfully with 4-bit quantization!


In [9]:
import torch
document = input("Enter your document text for paraphrasing:\n")

def split_into_chunks(text, max_tokens=750):
    tokens = tokenizer(text, return_tensors="pt", truncation=False).input_ids[0]
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))
    return chunks

paraphrased_chunks = []
chunks = split_into_chunks(document)

for i, chunk in enumerate(chunks):
    if i > 0:
        chunk = chunks[i-1][-100:] + "\n" + chunk

    prompt = (
           "You are an AI assistant skilled in paraphrasing. "
            "Your task is to rephrase the given text while preserving its original style, tone, and nuance.\n"
            "Return ONLY the paraphrased text without any explanations, prefixes, or suffixes.\n\n"
            f"### Original Text:\n{chunk}\n\n"
            "### Paraphrased Text:"
        )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=600).to(device)
    output = model.generate(
        **inputs,
        max_length=1000,
        do_sample=True,
        temperature=1,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

    paraphrased = tokenizer.decode(output[0], skip_special_tokens=True)
    paraphrased = paraphrased.split("Paraphrased Text:")[-1].strip()
    paraphrased_chunks.append(paraphrased)

paraphrased_document = "\n".join(paraphrased_chunks)

print("\n--- Paraphrased Document ---\n")
print(paraphrased_document)

Enter your document text for paraphrasing:
In the vast tapestry of the universe, where galaxies swirl in spirals of cosmic ballet and stars are born and extinguished in cataclysmic brilliance, humankind finds itself perched on a small, pale blue dot suspended in a sunbeam. Our world, a fragile orb teeming with life, floats in the immensity of space, surrounded by silent stars and the cold, unfeeling void. Despite our seemingly insignificant presence in the cosmos, our minds possess an insatiable curiosity—a relentless drive to explore, understand, and question the very fabric of reality.  Science, the great tool of inquiry, has carried us far. From the discovery of subatomic particles to the observation of distant quasars, we have unraveled secrets once hidden in the darkest corners of ignorance. Yet, as each question is answered, new ones arise, cascading like ripples on a pond. What lies beyond the observable universe? Are there other dimensions folded into the creases of spacetime? 

# **Llama 8B**

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision
    bnb_4bit_compute_dtype="float16",  # Compute with float16
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    quantization_config=bnb_config,
    device_map="auto"

print("Model loaded successfully with 4-bit quantization!")

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model loaded successfully with 4-bit quantization!


In [30]:
import torch
document = input("Enter your document text for paraphrasing:\n")

def split_into_chunks(text, max_tokens=750):
    tokens = tokenizer(text, return_tensors="pt", truncation=False).input_ids[0]
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))
    return chunks

paraphrased_chunks = []
chunks = split_into_chunks(document)

for i, chunk in enumerate(chunks):
    if i > 0:
        chunk = chunks[i-1][-100:] + "\n" + chunk

    prompt = (
           "You are an AI assistant skilled in paraphrasing. "
            "Your task is to rephrase the given text while preserving its original style, tone, and nuance.\n"
            "Return ONLY the paraphrased text without any explanations, prefixes, or suffixes.\n\n"
            f"### Original Text:\n{chunk}\n\n"
            "### Paraphrased Text:"
        )
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=600).to(device)
    output = model.generate(
        **inputs,
        max_length=1000,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    paraphrased = tokenizer.decode(output[0], skip_special_tokens=True)
    paraphrased = paraphrased.split("Paraphrased Text:")[-1].strip()
    paraphrased_chunks.append(paraphrased)

paraphrased_document = "\n".join(paraphrased_chunks)

print("\n--- Paraphrased Document ---\n")
print(paraphrased_document)

Enter your document text for paraphrasing:
In the vast tapestry of the universe, where galaxies swirl in spirals of cosmic ballet and stars are born and extinguished in cataclysmic brilliance, humankind finds itself perched on a small, pale blue dot suspended in a sunbeam. Our world, a fragile orb teeming with life, floats in the immensity of space, surrounded by silent stars and the cold, unfeeling void. Despite our seemingly insignificant presence in the cosmos, our minds possess an insatiable curiosity—a relentless drive to explore, understand, and question the very fabric of reality.  Science, the great tool of inquiry, has carried us far. From the discovery of subatomic particles to the observation of distant quasars, we have unraveled secrets once hidden in the darkest corners of ignorance. Yet, as each question is answered, new ones arise, cascading like ripples on a pond. What lies beyond the observable universe? Are there other dimensions folded into the creases of spacetime? 

# **streamlit**

In [31]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision
    bnb_4bit_compute_dtype="float16",  # Compute with float16
    bnb_4bit_use_double_quant=True,  # Further reduce memory usage
)

# Load the tokenizer and model with quantization
@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-3.1-8B-Instruct",
        quantization_config=bnb_config,
        device_map="auto"  # Automatically distributes model across available GPUs
    )
    return tokenizer, model

tokenizer, model = load_model()

# Streamlit App Title
st.title("📝 Paraphrasing Assistant with LLaMA 🤖")

# Paraphrasing Style Selection
paraphrase_style = st.selectbox(
    "Choose Paraphrasing Style:",
    ["Balanced", "Concise", "Creative"]
)

# Adjust parameters based on style
if paraphrase_style == "Balanced":
    temperature = 0.7
    top_p = 0.9
elif paraphrase_style == "Concise":
    temperature = 0.5
    top_p = 0.85
elif paraphrase_style == "Creative":
    temperature = 0.9
    top_p = 0.95

# User input for text to paraphrase
user_input = st.text_area("Enter your text to paraphrase:", height=200)

# Paraphrasing logic
if st.button("Paraphrase"):
    if user_input:
        with st.spinner("Paraphrasing..."):
            # Enhanced prompt
            prompt = (
                "You are an advanced AI assistant skilled in paraphrasing. "
                "Your task is to rephrase the given text while:\n"
                "- Preserving its original style, tone, and meaning.\n"
                "- Avoiding unnecessary verbosity or over-expansion.\n"
                "- Ensuring the output is natural and flows smoothly.\n\n"
                f"Original Text:\n{user_input}\n\n"
                "Paraphrased Text:"
            )

            # Tokenize input and move to device
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=600).to(model.device)
            output = model.generate(
                **inputs,
                max_length=1000,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                repetition_penalty=1.2,
                pad_token_id=tokenizer.eos_token_id
            )
            paraphrased = tokenizer.decode(output[0], skip_special_tokens=True)
            paraphrased = paraphrased.split("Paraphrased Text:")[-1].strip()

            st.success("Paraphrasing Complete!")
            st.write("### Paraphrased Text:")
            st.write(paraphrased)
    else:
        st.warning("Please enter text to paraphrase.")

# Footer
st.markdown("---")
st.markdown("Built using **Streamlit** and **LLaMA-3.1-8B-Instruct** with **4-bit Quantization**.")


Writing app.py


In [ ]:
# Install necessary libraries
!pip install transformers streamlit ngrok pyngrok --quiet
!ngrok config add-authtoken 2qpruL9I6szooqm9VpbJxQ0st1c_6XFijP2MQJRMW5PqRFzXq

# Start Streamlit on port 8501
!streamlit run app.py --server.port=8501 --server.address=0.0.0.0 &>/dev/null &

# Start ngrok tunnel
!ngrok http 8501 --log=stdout